In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


In [4]:
data_yelp = pd.read_csv('yelp_labelled.txt',sep='\t',header=None)

In [5]:
data_yelp.head()

,0,1
0,Wow... Loved this place.,1
1,Crust is not good.,0
2,Not tasty and the texture was just nasty.,0
3,Stopped by during the late May bank holiday of...,1
4,The selection on the menu was great and so wer...,1


In [9]:
column_name=['Review','Sentiment']
data_yelp.columns = column_name

In [10]:
data_yelp.head

<bound method NDFrame.head of                                                 Review  Sentiment
0                             Wow... Loved this place.          1
1                                   Crust is not good.          0
2            Not tasty and the texture was just nasty.          0
3    Stopped by during the late May bank holiday of...          1
4    The selection on the menu was great and so wer...          1
..                                                 ...        ...
995  I think food should have flavor and texture an...          0
996                           Appetite instantly gone.          0
997  Overall I was not impressed and would not go b...          0
998  The whole experience was underwhelming, and I ...          0
999  Then, as if I hadn't wasted enough of my life ...          0

[1000 rows x 2 columns]>

In [8]:
data_yelp.shape

(1000, 2)

In [11]:
data_amazon = pd.read_csv('amazon_cells_labelled.txt',sep='\t',header=None)

In [12]:
data_amazon.columns = column_name

In [13]:
data_amazon.head(
    
)

,Review,Sentiment
0,So there is no way for me to plug it in here i...,0
1,"Good case, Excellent value.",1
2,Great for the jawbone.,1
3,Tied to charger for conversations lasting more...,0
4,The mic is great.,1


In [14]:
data_amazon.shape

(1000, 2)

In [15]:
data_imdb = pd.read_csv('imdb_labelled.txt',sep='\t',header=None)


In [16]:
data_imdb.columns = column_name

In [18]:
data_imdb.head()

,Review,Sentiment
0,"A very, very, very slow-moving, aimless movie ...",0
1,Not sure who was more lost - the flat characte...,0
2,Attempting artiness with black & white and cle...,0
3,Very little music or anything to speak of.,0
4,The best scene in the movie was when Gerardo i...,1


In [19]:
data_imdb.shape

(748, 2)

In [20]:
data= data_yelp.append([data_amazon, data_imdb],ignore_index=True)

In [21]:
data.shape

(2748, 2)

In [22]:
data.head()

,Review,Sentiment
0,Wow... Loved this place.,1
1,Crust is not good.,0
2,Not tasty and the texture was just nasty.,0
3,Stopped by during the late May bank holiday of...,1
4,The selection on the menu was great and so wer...,1


In [23]:
data['Sentiment'].value_counts()

1    1386
0    1362
Name: Sentiment, dtype: int64

In [24]:
data.isnull().sum()

Review       0
Sentiment    0
dtype: int64

In [25]:
x=data['Review']
y=data['Sentiment']

DATA CLEANING

In [35]:
import string
import spacy


In [27]:
punct = string.punctuation

In [28]:
punct

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [29]:
from spacy.lang.en.stop_words import STOP_WORDS

In [30]:
stopwords = list(STOP_WORDS)

In [36]:
nlp=spacy.load('en_core_web_sm')
def text_data_cleaning(sentence):
  doc = nlp(sentence)

  tokens =[]
  for token in doc:
    if token.lemma_!="-PRON-":
      temp= token.lemma_.lower().strip()
    else:
      temp = token.lower_
    tokens.append(temp)
  
  cleaned_tokens =[]
  for token in tokens:
    if token not in stopwords and token not in punct:
      cleaned_tokens.append(token)

  return cleaned_tokens

In [37]:
text_data_cleaning("Hello all, It's a beautiful day outside there!")

['hello', 'beautiful', 'day', 'outside']

VECOTORIZATION FEATURE ENGINEERING (TF-IDF)


In [39]:
from sklearn.svm import LinearSVC
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline

In [40]:
tfidf = TfidfVectorizer(tokenizer=text_data_cleaning)

In [41]:
classifier= LinearSVC()

TRAIN THE MODEL

In [43]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x,y, test_size=0.2, random_state=0)

In [44]:
x_train.shape, x_test.shape

((2198,), (550,))

In [46]:
x_train.head()

2572    An Italian reviewer called this "a small, grea...
526                          And it was way to expensive.
1509    As an earlier review noted, plug in this charg...
144     Nice blanket of moz over top but i feel like t...
2483    The film gives meaning to the phrase, "Never i...
Name: Review, dtype: object

In [52]:
clf = Pipeline ([('tfidf',tfidf),('clf',classifier)])

In [53]:
clf.fit(x_train,y_train)

Pipeline(steps=[('tfidf',
                 TfidfVectorizer(tokenizer=<function text_data_cleaning at 0x7f2c47e05c10>)),
                ('clf', LinearSVC())])

PREDICT THE TEST SET RESULTS

In [56]:
from sklearn.metrics import accuracy_score,classification_report,confusion_matrix

In [57]:
y_pred= clf.predict(x_test)

In [58]:
confusion_matrix(y_test, y_pred)

array([[200,  79],
       [ 51, 220]])

In [59]:
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.80      0.72      0.75       279
           1       0.74      0.81      0.77       271

    accuracy                           0.76       550
   macro avg       0.77      0.76      0.76       550
weighted avg       0.77      0.76      0.76       550



In [60]:
accuracy_score(y_test,y_pred)

0.7636363636363637

In [61]:
clf.predict(["Wow, I am learning Natural Language Processing in fun fashion!"])

array([1])

In [62]:
clf.predict(["It's hard to learn new things!"])

array([0])